# Preparing the data

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds



C:\tools\Anaconda3\envs\MNIST\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: partially initialized module 'charset_normalizer' has no attribute 'md__mypyc' (most likely due to a circular import)

In [8]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]

In [5]:
BUFFER_SIZE = 70_000
BATCH_SIZE = 128
NUM_EPOCHS = 20

In [3]:
mnist_dataset, mnist_info =tfds.load(name='mnist', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/2 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/3 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|          | 0/4 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|          | 0/1 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Extraction completed...:   0%|          | 0/1 [00:00<?, ? file/s]

Dl Size...:   0%|          | 0/10 [00:00<?, ? MiB/s]

Dl Size...:   0

Generating train examples...: 52502 examples [00:15, 3420.50 examples/s]
Generating train examples...: 52845 examples [00:15, 3407.37 examples/s]
Generating train examples...: 53186 examples [00:15, 3402.33 examples/s]
Generating train examples...: 53532 examples [00:15, 3413.68 examples/s]
Generating train examples...: 53876 examples [00:15, 3415.67 examples/s]
Generating train examples...: 54218 examples [00:16, 3390.99 examples/s]
Generating train examples...: 54563 examples [00:16, 3402.71 examples/s]
Generating train examples...: 54906 examples [00:16, 3405.01 examples/s]
Generating train examples...: 55253 examples [00:16, 3418.53 examples/s]
Generating train examples...: 55603 examples [00:16, 3436.94 examples/s]
Generating train examples...: 55947 examples [00:16, 3431.95 examples/s]
Generating train examples...: 56292 examples [00:16, 3431.46 examples/s]
Generating train examples...: 56636 examples [00:16, 3417.97 examples/s]
Generating train examples...: 56978 examples [00:16

Dataset mnist downloaded and prepared to C:\Users\007303173\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

The scale method will make all values be between 0 and 1

In [5]:
def scale(image, label):
  image= tf.cast(image, tf.float32)
  image /= 225.
  return image, label

In [6]:
train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

Split the data for testing and validation

In [7]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [8]:
num_test_samples = 0.1 * mnist_info.splits['train'].num_examples
num_test_samples= tf.cast(num_test_samples, tf.int64)

BUFFER_SIZE is the size opf the batch being shuffles in this case its the entire dataset

In [9]:
train_and_validation_data = train_and_validation_data.shuffle(BUFFER_SIZE)

In [10]:
train_data = train_and_validation_data.skip(num_validation_samples)
validation_data = train_and_validation_data.take(num_validation_samples)

In [11]:
train_data = train_data.batch(BATCH_SIZE)

The validation and test sets dont need to be batched. Since we dont backward propiagate on them, however the model expects them to be batched

In [12]:
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

# Creating the Model

1.  The fisrt layer has a 5x5 kernel it produces 50 feature maps.
2.  The second layer has a 3x3 kernel it produces 50 feature maps
3.  The dense layer output is 10



In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(50, 5, activation = 'relu', input_shape = (28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.Conv2D(50, 3, activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size = (2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])

In [14]:
model.summary(line_length = 75)

Model: "sequential"
___________________________________________________________________________
 Layer (type)                    Output Shape                  Param #     
 conv2d (Conv2D)                 (None, 24, 24, 50)            1300        
                                                                           
 max_pooling2d (MaxPooling2D)    (None, 12, 12, 50)            0           
                                                                           
 conv2d_1 (Conv2D)               (None, 10, 10, 50)            22550       
                                                                           
 max_pooling2d_1 (MaxPooling2D)  (None, 5, 5, 50)              0           
                                                                           
 flatten (Flatten)               (None, 1250)                  0           
                                                                           
 dense (Dense)                   (None, 10)                    12510

In [15]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

In [16]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

In [17]:
early_stopping =tf.keras.callbacks.EarlyStopping(
  monitor= 'val_loss',
  mode = 'auto',
  min_delta = 0, 
  patience = 2,
  verbose = 0,
  restore_best_weights=True
)

In [18]:
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [26]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [22]:
model.fit(
    train_data,
    epochs = NUM_EPOCHS,
    callbacks = [tensorboard_callback,early_stopping],
    validation_data =validation_data,
    verbose=2
)

Epoch 1/20
422/422 - 51s - loss: 0.0314 - accuracy: 0.9904 - val_loss: 0.0346 - val_accuracy: 0.9890 - 51s/epoch - 120ms/step
Epoch 2/20
422/422 - 61s - loss: 0.0275 - accuracy: 0.9912 - val_loss: 0.0230 - val_accuracy: 0.9928 - 61s/epoch - 143ms/step
Epoch 3/20
422/422 - 62s - loss: 0.0242 - accuracy: 0.9924 - val_loss: 0.0204 - val_accuracy: 0.9932 - 62s/epoch - 148ms/step
Epoch 4/20


KeyboardInterrupt: 

# Testing the Model

In [19]:
test_loss, test_accuracy =  model.evaluate(test_data)

2/2 [==============================] - 1s 243ms/step - loss: 2.3155 - accuracy: 0.1134


In [22]:
print('Test loss: {0:.4f}. Test accuracy: {1: .2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.0271. Test accuracy:  99.18%


Plotting Images and results 

In [1]:
import matplotlib.pyplot as plt
import numpy as np

Split the test_data into 2 arrays, containing the images and correponding labels

In [23]:
for images, labels in test_data.take(1):
    images_test = images.numpy()
    labels_test = labels.numpy()

Reshape the images into 28X28 form, suitable for matplotlib (original dimensions: (28x28x1)

In [24]:
images_plot = np.reshape(images_test, (10000,28,28))

ValueError: ignored

The image to be displayed and tested

In [ ]:
i = 1
plt.figures(figsize=(2,2))
plt.axis('off')
plt.imshow(images_plot[i-1],cmap='gray', aspect="auto")
plt.show
print("Label: {}".format(labels_test[i-1]))

Obtain the model's prediction (logits)

In [ ]:
predictions = model.predict(images_test[i+1:i])
probabilities = tf.nn,softmax

# Visualizing in tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "logs/dir"